# Installing Required Libraries & Importing Modules

In [ ]:
!pip install pytesseract pdf2image pillow json5 pandas tqdm psycopg2-binary python-dotenv transformers huggingface_hub torch accelerate datasets pymongo pymupdf pdfplumber PyPDF2 pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 106.7 MB/s eta 0:00:0000:0100:01
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=c0bf8fe8a36aac5802eecaf8cf7c53ceeaf783ddb3940979f7323e1f83f5f920
  Stored in directory: /root/.cache/pip/wheels/ba/5e/16/6117f8fe7e9c0c161a795e10d94645ebcf301ccbd01f66d8ec
Successfully built iopath


In [ ]:
# Windows-compatible poppler installation
import platform
import os
import urllib.request
import zipfile
from pathlib import Path

if platform.system() == 'Windows':
    print("Setting up poppler for Windows...")
    
    # Check if poppler is already available
    poppler_path = r"C:\Program Files\poppler\bin"
    if not os.path.exists(poppler_path):
        print("Poppler not found in standard location. Checking environment...")
        # For Windows, we'll assume poppler-utils is installed or use pdf2image without it
        print("Note: If PDF processing fails, please install poppler-utils for Windows")
        print("Download from: https://github.com/oschwartz10612/poppler-windows/releases/")
    else:
        print(f"Poppler found at: {poppler_path}")
        # Add to PATH if not already there
        if poppler_path not in os.environ.get('PATH', ''):
            os.environ['PATH'] = poppler_path + ';' + os.environ.get('PATH', '')
            
print("Poppler setup complete")

In [3]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 32.7 MB/s eta 0:00:00:00:0100:01


In [54]:
import os
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import re
import json5
import pandas as pd
from tqdm import tqdm
import psycopg2
from dotenv import load_dotenv
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from huggingface_hub import login
from pymongo import MongoClient
import json
import torch
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import logging

# Loading Environment Variables

In [ ]:
load_dotenv()

# Hugging Face Token (optional)
hf_token = os.getenv('HF_TOKEN')
if hf_token:
    print("Hugging Face token loaded")
else:
    print("No Hugging Face token found - some features may be limited")

# MongoDB Configuration (optional)
mongo_host = os.getenv('MONGO_HOST')
mongo_port = os.getenv('MONGO_PORT')
mongo_db = os.getenv('MONGO_DB')
mongo_collection = os.getenv('MONGO_COLLECTION')

# PostgreSQL Configuration (with defaults)
postgres_host = os.getenv('POSTGRES_HOST', 'localhost')
postgres_port = os.getenv('POSTGRES_PORT', '5432')
postgres_db = os.getenv('POSTGRES_DB', 'pfe_db')
postgres_user = os.getenv('POSTGRES_USER', 'postgres')
postgres_password = os.getenv('POSTGRES_PASSWORD', '1986')

print(f"PostgreSQL config: {postgres_host}:{postgres_port}/{postgres_db}")
print(f"MongoDB config: {mongo_host}:{mongo_port}/{mongo_db} (collection: {mongo_collection})")

# Hugging Face Connection

In [ ]:
# Hugging Face Login (optional)
if hf_token:
    try:
        login(hf_token)
        print("Hugging Face login successful")
    except Exception as e:
        print(f"Hugging Face login failed: {e}")
        print("Continuing without HF authentication - some features may be limited")
else:
    print("Skipping Hugging Face login - no token provided")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


# MongoDB Connection

In [ ]:
# MongoDB Connection with error handling
client = None
db = None
resumes = None

try:
    # Check if MongoDB variables are properly set
    if mongo_host and mongo_port and mongo_db and mongo_collection:
        print(f"Connecting to MongoDB at {mongo_host}:{mongo_port}")
        client = MongoClient(f"{mongo_host}:{mongo_port}")
        db = client[mongo_db]
        resumes = db[mongo_collection]
        print("MongoDB connection successful")
    else:
        print("MongoDB configuration incomplete. Skipping MongoDB connection.")
        print(f"MONGO_HOST: {mongo_host}, MONGO_PORT: {mongo_port}")
        print(f"MONGO_DB: {mongo_db}, MONGO_COLLECTION: {mongo_collection}")
except Exception as e:
    print(f"MongoDB connection failed: {e}")
    print("Continuing without MongoDB. Results will only be saved to PostgreSQL.")
    client = None
    db = None
    resumes = None

# Postgres Connection

In [18]:
def get_postgres_connection():
    try:
        conn = psycopg2.connect(
            dbname=postgres_db,
            user=postgres_user,
            password=postgres_password,
            host=postgres_host,
            port=postgres_port
        )
        print("✅ Connected to PostgreSQL database.")
        return conn
    except Exception as e:
        print(f"❌ Error connecting to PostgreSQL: {e}")
        return None

# Loading Model and Tokenizer

In [1]:
# Set the path where you want to store the model
drive_model_path = "/content/drive/MyDrive/PFE/ai_models/mixtral-8x7b"

# Check if model is already downloaded to Drive
if os.path.exists(drive_model_path):
    print("Loading model from Drive cache...")
    tokenizer = AutoTokenizer.from_pretrained(drive_model_path)
    model = AutoModelForCausalLM.from_pretrained(
        drive_model_path,
        device_map="auto"
    )
else:
    print("Downloading model from HuggingFace...")
    MODEL_NAME = "mistralai/Mixtral-8x7B-Instruct-v0.1"

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto"
    )

    # Save to Drive for future use
    print("Saving model to Google Drive for reuse...")
    tokenizer.save_pretrained(drive_model_path)
    model.save_pretrained(drive_model_path)

# Set padding token if needed
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Create pipeline
chat_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    pad_token_id=tokenizer.pad_token_id,
    return_full_text=False
)


💾 Available RAM: 2.4 GB
🖥️  GPU Available: False
⚡ System resources limited - using lightweight extraction
🪶 Loading lightweight extraction model (DistilBERT-based)...


💾 Available RAM: 2.4 GB
🖥️  GPU Available: False
⚡ System resources limited - using lightweight extraction
🪶 Loading lightweight extraction model (DistilBERT-based)...


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

💾 Available RAM: 2.4 GB
🖥️  GPU Available: False
⚡ System resources limited - using lightweight extraction
🪶 Loading lightweight extraction model (DistilBERT-based)...


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


✅ Lightweight model (distilgpt2) loaded successfully!
📝 Note: Using rule-based extraction with AI assistance for better results

🎯 Using: Lightweight AI model (distilgpt2) + rule-based extraction


# Text extraction (OCR or PyMuPDF)

In [52]:
def extract_text(pdf_path):
    try:
        # Try PDFPlumber first (more accurate than PyMuPDF for many layouts)
        full_text = ""
        logging.getLogger("pdfminer").setLevel(logging.ERROR)
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    full_text += text + "\n"

        full_text = full_text.strip()

        # If text is empty or too short, fallback to OCR
        if not full_text or len(full_text) < 50:
            print("🔁 Switching to OCR (image-based or low-quality PDF)")
            images = convert_from_path(pdf_path)
            full_text = ""
            for i, img in enumerate(images):
                ocr_result = pytesseract.image_to_string(img)
                full_text += ocr_result + "\n"

        return full_text.strip()

    except Exception as e:
        print(f"❌ Error during text extraction: {e}")
        return None

# Clean up the text

In [44]:
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Prompt creation


In [12]:
def create_prompt(resume_text):
    prompt = f"""
Extract structured data from the following resume text and return a valid JSON object with the fields:

- name, age, email, phone, personal_links (github, linkedin, portfolio, indeed, others), skills, languages, experiences (job_title, company, duration, description), projects (project_title, project_summary), education (degree, institution, year), certifications, years_of_experience.

Return only a **raw JSON object**. Do NOT include any explanation, markdown, or code block (like ```json or ```python). Just the JSON like this example (no comments or extra text):
Example:
{{
  "name": "Jane Doe",
  "age": 28,
  "email": "jane.doe@example.com",
  "phone": "+212612345678",
  "personal_links": {{
    "github": "https://github.com/janedoe",
    "linkedin": "https://linkedin.com/in/janedoe",
    "portfolio": "https://janedoe.dev",
    "indeed": "https://indeed.com/profile/janedoe",
    "others": ["https://behance.net/janedoe", "https://medium.com/@janedoe"]
  }},
  "skills": ["Python", "React", "MongoDB", "Docker", "problem solving", "communication"],
  "languages": ["English", "Arabic", "French"],
  "experiences": [
    {{
      "job_title": "AI Engineer",
      "company": "TechCorp",
      "duration": "Jan 2020 - Present",
      "description": "Led a team of engineers to develop NLP models for document parsing."
    }}
  ],
  "projects": [
    {{
      "project_title": "Resume Parser",
      "project_summary": "Developed a parser using Transformers and MongoDB."
    }}
  ],
  "education": [
    {{
      "degree": "BSc in Computer Engineering",
      "institution": "ENSA",
      "year": "2022"
    }}
  ],
  "certifications": ["AWS Certified ML Engineer"],
  "years_of_experience": 5
}}

Resume Text:
{resume_text}
"""
    return prompt

# Call Hugging Face model

In [2]:
# Call Hugging Face model
def call_AI_model(prompt):
    output = chat_pipeline(
        prompt,
        max_new_tokens=1200,
        do_sample=True,
        temperature=0.2,
        return_full_text=False
    )[0]['generated_text']
    return output

# Parse model response

In [14]:
def parse_AI_model_output(ai_output):
    try:
        json_str = ai_output.strip().strip('```json').strip('```').strip()
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        return None

# Save to MongoDB

In [15]:
def save_to_mongodb(resume):
    try:
        resumes.insert_one(resume)
        print("✅ Saved to MongoDB!")
    except Exception as e:
        print(f"❌ Error saving to MongoDB: {e}")

# Full resume processing

In [25]:
def process_resume(pdf_path, uuid=None):
    resume_text = extract_text(pdf_path)
    if not resume_text:
        print("❌ No text found in resume. Skipping.")
        return False

    resume_text = preprocess_text(resume_text)
    print("📄 Extracted and preprocessed text:", resume_text)
    prompt = create_prompt(resume_text)
    AI_model_output = call_AI_model(prompt)
    print("ai model output:", AI_model_output)
    parsed_data = parse_AI_model_output(AI_model_output)
    print("parsed data output:", parsed_data)

    if parsed_data:
        parsed_data['uuid'] = uuid
        save_to_mongodb(parsed_data)
        print(f"✅ Successfully processed and stored resume UUID {uuid}")
        return True
    else:
        print("❌ Failed to parse resume.")
        return False

# Process all resumes from Postgres database

In [ ]:
def update_parsed_flag(resume_id):
    try:
        conn = get_postgres_connection()
        cursor = conn.cursor()
        cursor.execute("UPDATE resumes SET parsed = TRUE WHERE id = %s;", (resume_id,))
        conn.commit()
        conn.close()
        return True
    except Exception as e:
        print(f"❌ Failed to update resume ID {resume_id}: {e}")
        return False

In [19]:
def process_all_resumes_from_db():
    conn = get_postgres_connection()
    cursor = conn.cursor()

    cursor.execute("SELECT resume_id, uuid, storage_path FROM resumes WHERE parsed = FALSE;")

    failed_resumes = []
    rows = cursor.fetchall()

    if not rows:
        print("✅ No new resumes to process.")
        return

    for row in tqdm(rows, desc="Processing Resumes"):
        resume_id, uuid, file_path = row
        print(f"\n🔍 Processing resume ID {resume_id}: {file_path}")
        try:
            success = process_resume(file_path, uuid)
            if success:
                update_parsed_flag(resume_id)
        except Exception as e:
            print(f"❌ Unexpected error while processing {file_path}: {e}")
            failed_resumes.append(file_path)

    conn.close()

    if failed_resumes:
        print("\n⚠️ Failed to process the following resumes:")
        for path in failed_resumes:
            print(f" - {path}")

# Run the processing function

In [53]:
process_all_resumes_from_db()

✅ Connected to PostgreSQL database.


Processing Resumes:   0%|          | 0/28 [00:00<?, ?it/s]WARNING:pdfminer.pdfpage:CropBox missing from /Page, defaulting to MediaBox



🔍 Processing resume ID 12: /content/drive/MyDrive/PFE/resumes_dataset/20.pdf


📄 Extracted and preprocessed text: CURRICULUM VITAE CONTACT INFORMATION: Surili Chawla +91-2382257665 chawlasurili@gmail.com EDUCATION: Qualification/Degree School/College University or Board Year Percent age/ CGPA Jaypee Institute of Jaypee Institute of 2019-2021 9.0 MTech (CSE with Information Information Technology specialization in Technology, Noida Data Analytics) Chitkara University 2014-2018 8.1 Bachelor of Chitkara School Of Engineering Engg. & Tech., (CSE) Himachal Pradesh WORK EXPERIENCE: Infosys Limited, Chandigarh (July,2021-Present) I worked on Data Visualization, Data Analytics, Observability, Monitoring and Alerting as a Systems Engineer. My role revolved around understanding the client requirements and creating dashboards using the streaming data from data sources like elastic search, Prometheus and Thanos. Identifying and selecting best representations/visualization in Grafana to depict the required information helped the client in making critical business decisions wi

Processing Resumes:   4%|▎         | 1/28 [03:40<1:39:18, 220.70s/it]WARNING:pdfminer.pdfpage:CropBox missing from /Page, defaulting to MediaBox


...

Processing Resumes:  96%|█████████▋| 27/28 [1:00:27<01:16, 76.06s/it]

ai model output: 

{
  "name": "John W. Smith",
  "email": "jwsmith@colostate.edu",
  "skills": [],
  "experiences": [
    {
      "job_title": "Counseling Supervisor",
      "company": "The Wesley Center",
      "duration": "1999-2002",
      "description": ""
    },
    {
      "job_title": "Client Specialist",
      "company": "Rainbow Special Care Center",
      "duration": "1997-1999",
      "description": ""
    },
    {
      "job_title": "Teacher’s Assistant",
      "company": "Cowell Elementary",
      "duration": "1996-1997",
      "description": ""
    }
  ],
  "education": [
    {
      "degree": "BS in Early Childhood Development",
      "institution": "University of Arkansas at Little Rock",
      "year": "1999"
    },
    {
      "degree": "BA in Elementary Education",
      "institution": "University of Arkansas at Little Rock",
      "year": "1998"
    }
  ],
  "years_of_experience": 4
}
parsed data output: {'name': 'John W. Smith', 'email': 'jwsmith@colostate.edu', 's

📄 Extracted and preprocessed text: CHRISTOPER Summary Senior Web Developer specializing in front end development. MORGAN Experienced with all stages of the development cycle for dynamic web projects. Well-versed in numerous programming languages including HTML5, PHP OOP, JavaScript, CSS, MySQL. Strong background in project management and customer relations. Skill Highlights • Project management • Creative design • Strong decision maker • Innovative • Complex problem solver • Service-focused Experience Web Developer - 09/2015 to 05/2019 Luna Web Design, New York • Cooperate with designers to create clean interfaces and simple, intuitive interactions and experiences. • Develop project concepts and maintain optimal workflow. Contact • Work with senior developer to manage large, complex Address: design projects for corporate clients. 177 Great Portland Street, • Complete detailed programming and development tasks London W5W 6PQ for front end public and internal websites as well as challeng

Processing Resumes: 100%|██████████| 28/28 [1:01:33<00:00, 131.90s/it]

ai model output: 

{
  "name": "CHRISTOPER",
  "email": "christoper.m@gmail.com",
  "phone": "+44 (0)20 7666 8555",
  "personal_links": {
    "github": null,
    "linkedin": null,
    "portfolio": null,
    "indeed": null,
    "others": []
  },
  "skills": ["project management", "creative design", "strong decision maker", "innovative", "complex problem solver", "service-focused"],
  "languages": ["Spanish - C2", "Chinese - A1", "German - A2"],
  "experiences": [
    {
      "job_title": "Web Developer",
      "company": "Luna Web Design",
      "duration": "09/2015 to 05/2019",
      "description": "Cooperate with designers to create clean interfaces and simple, intuitive interactions and experiences. Develop project concepts and maintain optimal workflow. Complete detailed programming and development tasks for front end public and internal websites as well as challenging back-end server code. Carry out quality assurance tests to discover errors and optimize usability."
    }
  ],
  "p